## Setup

In [38]:
import tensorflow as tf
import numpy
import scipy.misc
import glob, os
import random

## Data parameters

In [39]:
IMG_CLASSES = ['red', 'green', 'blue']
NCLASSES = len(IMG_CLASSES)
NCHAN = 3
INPUT_X_SZ = 32
INPUT_Y_SZ = 32

train_test_ratio = 0.9

## Hyperparameters

In [40]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001

## Load Data

In [41]:
def get_data():
	BASE_PATH = 'traffic_light_imgs/'

	all_data = []
	for im_class in IMG_CLASSES:
		for img_path in glob.glob(os.path.join(BASE_PATH, im_class, "*")):
			img = scipy.misc.imread(img_path)
			assert img is not None
			img = scipy.misc.imresize(img, (INPUT_X_SZ, INPUT_Y_SZ))
			one_hot_vector = [0] * NCLASSES
			one_hot_vector[IMG_CLASSES.index(im_class)] = 1
			assert img.shape == (32, 32, 3)

			all_data.append((img, one_hot_vector))

	random.shuffle(all_data)

	split_idx = int(len(all_data) * train_test_ratio)

	# correct split_idx to be a multiple of batch sz
	split_idx -= split_idx % BATCH_SIZE
	end_idx = len(all_data) - ((len(all_data) - split_idx) % BATCH_SIZE)

	train_set = all_data[:split_idx]
	test_set = all_data[split_idx:end_idx]

	assert len(train_set) % BATCH_SIZE == 0
	assert len(test_set) % BATCH_SIZE == 0

	return train_set, test_set

train_set, test_set = get_data()

## Net Architecture

In [42]:
# . . . . . . . . . .      (input data, 1-deep)                 X   [batch, 32, 32, 3]
# @ @ @ @ @ @ @ @ @ @   -- conv. layer 5x5x1=>6 stride 1        c1f [5, 5, 3, 6]
# :::::::::::::::::::                                           C1  [batch, 32, 32, 6]
#						-- ReLU                                 R1
#						-- maxpool, 2x2 window, stride 2        S1  [batch, 16, 16, 6]
#   @ @ @ @ @ @ @ @     -- conv. layer 6x6x6=>4 stride 1        c2f [6, 6, 6, 4]
#   :::::::::::::::                                             C2  [batch, 16, 16, 4]
#						-- ReLU                                 R2
#						-- maxpool, 2x2 window, stride 2        S2  [batch, 8, 8, 4] => reshaped to S2r [batch, 8*8*4]
#      \x/x\x\x/        -- fully connected layer                fc1w[8*8*4, 100]
#       . . . .                                                 FC1 [batch, 100]
#						-- ReLU                                 R3
#       \x/x\x/         -- fully connected layer                fc2w[100, 3]
#        . . .													FC2
#            			-- softmax								pred[batch, 3]
#

# Model parameters
C1_NFILT = 6
C1_FILTSZ = 5
C2_NFILT = 4
C2_FILTSZ = 6
FC_NNODES = 100
FC1_INPUTSZ = 8*8*4

# Define Variables
X  = tf.placeholder(tf.float32, shape=[None, INPUT_X_SZ, INPUT_Y_SZ, NCHAN])
Y_ = tf.placeholder(tf.float32, shape=[None, NCLASSES])

c1f  = tf.Variable(tf.random_normal([C1_FILTSZ, C1_FILTSZ, NCHAN, C1_NFILT]))
c1b  = tf.Variable(tf.random_normal([C1_NFILT])) # correct bias for CONV?

c2f  = tf.Variable(tf.random_normal([C2_FILTSZ, C2_FILTSZ, C1_NFILT, C2_NFILT]))
c2b  = tf.Variable(tf.random_normal([C2_NFILT])) # correct bias for CONV?

fc1w = tf.Variable(tf.random_normal([FC1_INPUTSZ, FC_NNODES]))
fc1b = tf.Variable(tf.random_normal([FC_NNODES]))

fc2w = tf.Variable(tf.random_normal([FC_NNODES, NCLASSES]))
fc2b = tf.Variable(tf.random_normal([NCLASSES]))

In [43]:
# Define Model
C1 = tf.nn.conv2d(X, c1f, [1, 1, 1, 1], padding='SAME')
R1 = tf.nn.relu(C1+c1b)
S1 = tf.nn.max_pool(R1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

C2 = tf.nn.conv2d(S1, c2f, [1, 1, 1, 1], padding='SAME')
R2 = tf.nn.relu(C2+c2b)
S2 = tf.nn.max_pool(R2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

S2r = tf.reshape(S2, [-1, FC1_INPUTSZ])

FC1 = tf.matmul(S2r, fc1w)
R3 = tf.nn.relu(FC1 + fc1b)

FC2 = tf.matmul(R3, fc2w) + fc2b
pred = tf.nn.softmax(FC2)

In [44]:
# Define Loss
cr = tf.nn.softmax_cross_entropy_with_logits(FC2, Y_)
loss = tf.reduce_mean(cr)*100

# Define Accuracy
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Define optimizer
opt = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
train_step = opt.minimize(loss)

init = tf.global_variables_initializer()

## Run!

In [45]:
# Run!
with tf.Session() as sess:
	sess.run(init)

	for repeat in range(10):
		print "repeat", repeat
		for batch_num in range(len(train_set) / BATCH_SIZE):
			batch = train_set[batch_num*BATCH_SIZE : (batch_num+1)*BATCH_SIZE]
			batch_x = numpy.array([s[0] for s in batch])
			batch_y = [s[1] for s in batch]
			load = {
				X : batch_x,
				Y_: batch_y,
			}
			sess.run(train_step, feed_dict=load)

			if batch_num % 4 == 0:
				acc = sess.run(accuracy, feed_dict=load)
				print "Trng set accurcy:", acc

				batch = test_set[:BATCH_SIZE]
				batch_x = numpy.array([s[0] for s in batch])
				batch_y = [s[1] for s in batch]
				load = {
					X : batch_x,
					Y_: batch_y
				}
				acc = sess.run(accuracy, feed_dict=load)
				print "Test set accurcy:", acc, '\n'

# print batch_y[0]
# scipy.misc.imshow(batch_x[0])

repeat 0
Trng set accurcy: 0.34375
Test set accurcy: 0.3125 

Trng set accurcy: 0.28125
Test set accurcy: 0.3125 

Trng set accurcy: 0.375
Test set accurcy: 0.28125 

Trng set accurcy: 0.46875
Test set accurcy: 0.28125 

Trng set accurcy: 0.40625
Test set accurcy: 0.28125 

Trng set accurcy: 0.34375
Test set accurcy: 0.25 

Trng set accurcy: 0.15625
Test set accurcy: 0.21875 

Trng set accurcy: 0.125
Test set accurcy: 0.15625 

Trng set accurcy: 0.0625
Test set accurcy: 0.125 

Trng set accurcy: 0.21875
Test set accurcy: 0.21875 

repeat 1
Trng set accurcy: 0.375
Test set accurcy: 0.34375 

Trng set accurcy: 0.28125
Test set accurcy: 0.34375 

Trng set accurcy: 0.53125
Test set accurcy: 0.34375 

Trng set accurcy: 0.4375
Test set accurcy: 0.4375 

Trng set accurcy: 0.5625
Test set accurcy: 0.4375 

Trng set accurcy: 0.625
Test set accurcy: 0.46875 

Trng set accurcy: 0.78125
Test set accurcy: 0.5625 

Trng set accurcy: 0.75
Test set accurcy: 0.59375 

Trng set accurcy: 0.8125
Test set 